In [3]:
import sqlite3
import pandas as pd
from pathlib import Path

# Setup project root
current_dir = Path.cwd()
if "ENEXIS" in str(current_dir):
    while current_dir.name != "ENEXIS" and current_dir.parent != current_dir:
        current_dir = current_dir.parent
    project_root = current_dir
else:
    project_root = current_dir

# Create Naive Model with 7-day Lag
conn = sqlite3.connect(project_root / "src" / "data" / "WARP.db")
naive_model = pd.read_sql_query(
    "SELECT target_datetime, Price FROM master_warp ORDER BY target_datetime", 
    conn, parse_dates=['target_datetime']
).set_index('target_datetime')
conn.close()

naive_model['naive_price'] = naive_model['Price'].shift(168)
print(f"Naive model: {len(naive_model)} rows, {naive_model['naive_price'].count()} non-null predictions")

Naive model: 3624 rows, 3456 non-null predictions


In [4]:
import pandas as pd

csv_path = "../../../workspaces/sharell/random_forest_predictions_with_actuals.csv"

# 1. Read existing CSV
base_df = pd.read_csv(csv_path)

# 2. Convert to datetime with UTC
base_df['target_datetime'] = pd.to_datetime(base_df['target_datetime'], utc=True)
naive_model.index = pd.to_datetime(naive_model.index, utc=True)

# 3. Reset index to make target_datetime a column for merging
naive_df = naive_model.reset_index()

# 4. Merge only naive_price column
merged_df = base_df.merge(
    naive_df[['target_datetime', 'naive_price']],
    on=['target_datetime'],
    how='left'
)

# 5. Write back to CSV
merged_df.to_csv(csv_path, index=False)
print(f"✅ Naive price merged! Laatste regels:\n", merged_df.tail())

✅ Naive price merged! Laatste regels:
                        run_date           target_datetime  predicted_Price  \
4315  2025-04-13 00:00:00+00:00 2025-04-19 20:00:00+00:00         0.092347   
4316  2025-04-13 00:00:00+00:00 2025-04-19 21:00:00+00:00         0.100777   
4317  2025-04-13 00:00:00+00:00 2025-04-19 22:00:00+00:00         0.093272   
4318  2025-04-13 00:00:00+00:00 2025-04-19 23:00:00+00:00         0.091806   
4319  2025-04-13 00:00:00+00:00 2025-04-20 00:00:00+00:00         0.091289   

      actual_Price  oxygent_price  naive_price  
4315       0.11990          0.162      0.08924  
4316       0.10390          0.148      0.08200  
4317       0.09441          0.145      0.06990  
4318       0.09800          0.130      0.06399  
4319       0.09075          0.113      0.05600  
